In [1]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import bolift
from langchain.prompts.prompt import PromptTemplate
import copy, cloudpickle
import seaborn as sns

import os
from dotenv import load_dotenv
load_dotenv()

/Users/maykcaldas/miniconda3/envs/bolift/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
import urllib.request

urllib.request.urlretrieve(
    "https://github.com/google/fonts/raw/main/ofl/ibmplexmono/IBMPlexMono-Regular.ttf",
    "IBMPlexMono-Regular.ttf",
)
fe = font_manager.FontEntry(fname="IBMPlexMono-Regular.ttf", name="plexmono")
font_manager.fontManager.ttflist.append(fe)
plt.rcParams.update(
    {
        "axes.facecolor": "#f5f4e9",
        "grid.color": "#AAAAAA",
        "axes.edgecolor": "#333333",
        "figure.facecolor": "#FFFFFF",
        "axes.grid": False,
        "axes.prop_cycle": plt.cycler("color", plt.cm.Dark2.colors),
        "font.family": fe.name,
        "figure.figsize": (3.5, 3.5 / 1.2),
        "ytick.left": True,
        "xtick.bottom": True,
    }
)

import random
np.random.seed(0)
random.seed(0)

In [ ]:
# I saved the number of samples in the training set in the pickle file. With that, old pickle weren't compatible with the new printing format.
# This script adds an array [0 .. len(samples)] to the old pickle files, so that they are compatible with the new printing format.

import numpy as np
d = cloudpickle.load(open("./out/<OLD_FORMAT_BO>.pkl", "rb"))

data = d
for key, values in data.items():
    num_aqs = values.shape[0]
    num_entries = values.shape[1]
    new_values = np.empty((num_aqs, num_entries, 3), dtype='<U349')
    
    for j in range(num_aqs):
        for i in range(num_entries):
            description, value = values[j, i]
            new_values[j, i] = [description, str(i+1), value]
    data[key] = new_values

cloudpickle.dump(data, open("./out/<NEW_FORMAT_BO>.pkl", "wb"))

# BayesOpt Plot

In [3]:
def plot_BO(ax, data_file, title, data, axis_name, lim=None, label=False):
    d = cloudpickle.load(open(data_file, "rb"))
    N=20
    M=1

    for i in range(M):
        if "expected_improvement" in d.keys():
          ax.plot(
            [int(s) for s in d['expected_improvement'][i,:N, 1]],
            [float(y) for y in d['expected_improvement'][i,:N, 2]], 
            color="C1", alpha=0.1
          )
        if "greedy" in d.keys():
           ax.plot(
            [int(s) for s in d['greedy'][i,:N, 1]],
            [float(y) for y in d['greedy'][i,:N, 2]], 
            color="C2", alpha=0.1
          )
        if "upper_confidence_bound" in d.keys():
          ax.plot(
            [int(s) for s in d['upper_confidence_bound'][i,:N, 1]],
            [float(y) for y in d['upper_confidence_bound'][i,:N, 2]], 
            color="C3", alpha=0.1
          )
        if "probability_of_improvement" in d.keys():
          ax.plot(
            [int(s) for s in d['probability_of_improvement'][i,:N, 1]],
            [float(y) for y in d['probability_of_improvement'][i,:N, 2]], 
            color="C4", alpha=0.1
          )
        if "random" in d.keys():
          ax.plot(
            [int(s) for s in d['random'][i,:N, 1]],
            [float(y) for y in d['random'][i,:N, 2]], 
            color="C5", alpha=0.1
          )
    if "expected_improvement" in d.keys():
      label = "EI" if label else None
      ax.plot(
            d['expected_improvement'][:,:N, 1].astype('int').mean(axis=0),
            d['expected_improvement'][:,:N, 2].astype('float').mean(axis=0), 
            color="C1", label=label
          )
    if "greedy" in d.keys():
      label = "Greedy" if label else None
      ax.plot(
            d['greedy'][:,:N, 1].astype('int').mean(axis=0),
            d['greedy'][:,:N, 2].astype('float').mean(axis=0), 
            color="C2", label=label
          )
    if "upper_confidence_bound" in d.keys():
      label = "UCB" if label else None
      ax.plot(
            d['upper_confidence_bound'][:,:N, 1].astype('int').mean(axis=0),
            d['upper_confidence_bound'][:,:N, 2].astype('float').mean(axis=0), 
            color="C3", label=label
          )
    if "probability_of_improvement" in d.keys():
      label = "POI" if label else None
      ax.plot(
            d['probability_of_improvement'][:,:N, 1].astype('int').mean(axis=0),
            d['probability_of_improvement'][:,:N, 2].astype('float').mean(axis=0), 
            color="C4", label=label
          )
    if "random" in d.keys():
      label = "random" if label else None
      ax.plot(
            d['random'][:,:N, 1].astype('int').mean(axis=0),
            d['random'][:,:N, 2].astype('float').mean(axis=0), 
            color="C5", label=label
          )
    if "random_mean" in d.keys():
      label = "Random" if label else None
      ax.plot(
            d['random_mean'][:, :N, 0].astype('int').mean(axis=0),
            d['random_mean'][:, :N, 1].astype('float').mean(axis=0), 
            color="gray", label=label, linestyle="dashed"
      )
    ax.axhline(y=data.max(), color="C15", linestyle="--")
    ax.text(ax.get_xlim()[1]+1, data.max(), "max", va="center", ha="left", backgroundcolor="w", fontsize=8)
    ax.axhline(y=data.quantile(0.99), color="C14", linestyle="--")
    ax.text(ax.get_xlim()[1]+1, data.quantile(0.99), "99%", va="center", ha="left", backgroundcolor="w", fontsize=8)
    ax.axhline(y=data.quantile(0.95), color="C13", linestyle="--")
    ax.text(ax.get_xlim()[1]+1, data.quantile(0.95), "95%", va="center", ha="left", backgroundcolor="w", fontsize=8)
    ax.axhline(y=data.mean(), color="C12", linestyle="--")
    ax.text(ax.get_xlim()[1]+1, data.mean(), "mean", va="center", ha="left", backgroundcolor="w", fontsize=8)
    if not data_file[:3] == "sol":
      ax.axhline(y=data.quantile(0.05), color="C11", linestyle="--")
      ax.text(ax.get_xlim()[1]+1, data.quantile(0.05)+0.3, "5%", va="center", ha="left", backgroundcolor="w", fontsize=8)
      ax.axhline(y=data.min(), color="C10", linestyle="--")
      ax.text(ax.get_xlim()[1]+1, data.min()-0.3, "min", va="center", ha="left", backgroundcolor="w", fontsize=8)
    ax.set_title(title)

    ax.set_xlabel("Number of samples")
    ax.set_ylabel(f"Measured {axis_name}")
    # ax.set_xticks([i for i in range(0,N+1,5)], [str(x * 1) for x in [i for i in range(0,N+1,5)]])
    if lim:
      ax.set_ylim(lim)
    ax.set_title(title)

### C2

In [ ]:
np.random.seed(88)

data_path = "./paper/data/C2_yield_meth_oxy_short.csv"
# data_path = "./paper/data/ada_embedd_c2_dataset.csv"
raw_data = pd.read_csv(data_path, sep=";")
# raw_data = raw_data.sample(frac=1).reset_index(drop=True)

# raw_data['completion'] = - raw_data['completion']

x_name = "prompt"
y_name = "completion"



In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=4, figsize=(16,4), constrained_layout=True)
# for ax in axs.flat:
#     ax.set_aspect(0.6)

lim=(0,8)
raw_data = pd.read_csv("dataset/data/1180_ocm_dataset.csv", sep=";")
plot_BO(axs[0], "./out/C2_davinci_1180_1_tree.pkl","20", 
         raw_data[y_name], "C$_2$ yield", lim, label=True, data_file_random="./out/C2 - random - 1180.pkl")

lim=(0,10)
raw_data = pd.read_csv("dataset/data/2950_ocm_dataset.csv", sep=";")
plot_BO(axs[1], "./out/C2_davinci_2950_1_tree.pkl","50",
         raw_data[y_name], "C$_2$ yield", lim, label=False, data_file_random="./out/C2 - random - 2950.pkl")

lim=(0,25)
raw_data = pd.read_csv("dataset/data/5900_ocm_dataset.csv", sep=";")
plot_BO(axs[2], "./out/C2_davinci_5900_1_tree.pkl", "100",
         raw_data[y_name], "C$_2$ yield", lim, label=False, data_file_random="./out/C2 - random - 5900.pkl")

lim=(0,25)
raw_data = pd.read_csv("dataset/data/12744_ocm_dataset.csv", sep=";")
plot_BO(axs[3], "./out/C2_davinci_12744_1_tree_2.pkl", "216",
         raw_data[y_name], "C$_2$ yield", lim, label=False, data_file_random="./out/C2 - random - 12744.pkl")

fig.suptitle("TreePool with davinci")
fig.legend(loc='upper center', bbox_to_anchor=(0.5,0),
          fancybox=True, shadow=True, ncol=6)
plt.savefig(f"figs/BO_C2", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
pools = [
    './out/C2_davinci_1180_1_tree.pkl',
    './out/C2_davinci_2950_1_tree.pkl',
    './out/C2_davinci_5900_1_tree.pkl',
    './out/C2_davinci_12744_1_tree.pkl',
]
for p in pools:
    print(p)
    d = cloudpickle.load(open(p, "rb"))
    for run in d['upper_confidence_bound'][:, :, 0]:
        print([r[14:r.find(",")] for r in run])
    
    print()

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=4, figsize=(16,4), constrained_layout=True)
# for ax in axs.flat:
#     ax.set_aspect(0.6)

lim=(0,8)
raw_data = pd.read_csv("dataset/data/1180_ocm_dataset.csv", sep=";")
plot_BO(axs[0], "./out/C2_davinci_1180_1_16hh.pkl","20", 
         raw_data[y_name], "C$_2$ yield", lim, label=True, data_file_random="./out/C2 - random - 1180.pkl")

lim=(0,10)
raw_data = pd.read_csv("dataset/data/2950_ocm_dataset.csv", sep=";")
plot_BO(axs[1], "./out/C2_davinci_2950_1_16hh.pkl","50",
         raw_data[y_name], "C$_2$ yield", lim, label=False, data_file_random="./out/C2 - random - 2950.pkl")

lim=(0,25)
raw_data = pd.read_csv("dataset/data/5900_ocm_dataset.csv", sep=";")
plot_BO(axs[2], "./out/C2_davinci_5900_1_16hh.pkl", "100",
         raw_data[y_name], "C$_2$ yield", lim, label=False, data_file_random="./out/C2 - random - 5900.pkl")

lim=(0,25)
raw_data = pd.read_csv("dataset/data/12744_ocm_dataset.csv", sep=";")
plot_BO(axs[3], "./out/C2_davinci_12744_1_16hh.pkl", "216",
         raw_data[y_name], "C$_2$ yield", lim, label=False, data_file_random="./out/C2 - random - 12744.pkl")
         

fig.suptitle("Subpool of 16 with half random samples with davinci")
fig.legend(loc='upper center', bbox_to_anchor=(0.5,0),
          fancybox=True, shadow=True, ncol=6)
plt.savefig(f"figs/BO_C2", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
pools = [
    './out/C2_davinci_1180_1_16hh.pkl',
    './out/C2_davinci_2950_1_16hh.pkl',
    './out/C2_davinci_5900_1_16hh.pkl',
    './out/C2_davinci_12744_1_16hh.pkl',
]
for p in pools:
    print(p)
    d = cloudpickle.load(open(p, "rb"))
    for run in d['upper_confidence_bound'][:, :, 0]:
        print([r[14:r.find(",")] for r in run])
    
    print()

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=4, figsize=(16,4), constrained_layout=True)
# for ax in axs.flat:
#     ax.set_aspect(0.6)

lim=(0,25)
raw_data = pd.read_csv("dataset/data/12744_ocm_dataset.csv", sep=";")
plot_BO(axs[0], "./out/C2_davinci_fulldataset_new_subpool_16_allrandom_1init.pkl","all_random",
         raw_data[y_name], "C$_2$ yield", lim, label=True, data_file_random="./out/C2 - random - 12744.pkl")

plot_BO(axs[1], "./out/C2_davinci_12744_1_16hh.pkl","half_random",
         raw_data[y_name], "C$_2$ yield", lim, label=False, data_file_random="./out/C2 - random - 12744.pkl")

plot_BO(axs[2], "./out/C2_davinci_fulldataset_subpool_16_no_random_1_init.pkl", "no_random",
# plot_BO(axs[2], "./out/C2_davinci_fulldataset_subpool_16_no_random_newest_seed0_2_init.pkl", "no_random",
         raw_data[y_name], "C$_2$ yield", lim, label=False, data_file_random="./out/C2 - random - 12744.pkl")

plot_BO(axs[3], "./out/C2_davinci_12744_1_tree_2.pkl", "TreePool",
         raw_data[y_name], "C$_2$ yield", lim, label=False, data_file_random="./out/C2 - random - 12744.pkl")
         

fig.suptitle("216 samples with each subpool")
fig.legend(loc='upper center', bbox_to_anchor=(0.5,0),
          fancybox=True, shadow=True, ncol=6)
plt.savefig(f"figs/BO_C2", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
pools = [
    # './out/C2_davinci_fulldataset_new_subpool_16_allrandom_1init.pkl',
    # './out/C2_davinci_12744_1_16hh.pkl',
    # './out/C2_davinci_fulldataset_subpool_16_no_random_1_init.pkl',
    './out/C2_davinci_fulldataset_subpool_16_no_random_newest_seed0_2_init.pkl',
    # './out/C2_davinci_12744_1_tree.pkl',
]
for p in pools:
    print(p)
    d = cloudpickle.load(open(p, "rb"))
    for k in d.keys():
        for run in d[k][:, :, 0]:
            print(k, [r[14:r.find(",")] for r in run])
    
    print()

In [ ]:
d_davinci = cloudpickle.load(open("paper/out/C2_davinci_100.pkl", "rb"))
print(d_davinci['expected_improvement'][:, -1, 1].astype(float))
best_davinci = d_davinci['expected_improvement'][:, :, 1].astype(float).mean(axis=0)[-1]
print(f"DaVinci is top{np.sum(raw_data[y_name] > best_davinci)}: {best_davinci}")

d_gpt4 = cloudpickle.load(open("paper/out/C2_GPT4_100.pkl", "rb"))
print(d_gpt4['upper_confidence_bound'][:, -1, 1].astype(float))
best_gpt4 = d_gpt4['upper_confidence_bound'][:, :, 1].astype(float).mean(axis=0)[-1]
print(f"Gpt4 is top{np.sum(raw_data[y_name] > best_gpt4)}: {best_gpt4}")

d_gpr = cloudpickle.load(open("paper/out/C2_GPR_100.pkl", "rb"))
print(d_gpr['expected_improvement'][:, -1, 1].astype(float))
best_gpr = d_gpr['upper_confidence_bound'][:, :, 1].astype(float).mean(axis=0)[-1]
print(f"GPR is top{np.sum(raw_data[y_name] > best_gpr)}: {best_gpr}")

sns.histplot(raw_data[y_name])
# print(np.sum(raw_data[y_name] > best))
plt.xlabel("measured C$_2$ yield")
plt.axvline(best_davinci, color='C1', linestyle='--', label="Davinci")
plt.axvline(best_gpt4, color='C2', linestyle='--', label="GPT4")
plt.axvline(best_gpr, color='C3', linestyle='--', label="GPR")
plt.legend()
plt.savefig(f"figs/hist_C2", dpi=300, bbox_inches='tight')
plt.show()

print(raw_data[raw_data[y_name] > best_davinci])



### iupac-sol

In [ ]:
np.random.seed(0)
data_path = "paper/data/esol_iupac.csv"
raw_data = pd.read_csv(data_path)
raw_data = raw_data.dropna()
raw_data = raw_data[["IUPAC", "measured log(solubility:mol/L)"]]
raw_data = raw_data.sample(frac=1).reset_index(drop=True)

# raw_data['measured log(solubility:mol/L)'] = -raw_data['measured log(solubility:mol/L)']
x_name = "IUPAC"
y_name = "measured log(solubility:mol/L)"

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(12,4), constrained_layout=True)
for ax in axs.flat:
    ax.set_aspect(1.8)

lim=(-5.5,2)

plot_BO(axs[0], "paper/out/sol_davinci_100.pkl", "Topk - davinci",
         raw_data[y_name], "LogS solubility", lim, label=False, data_file_random="paper/out/sol - random.pkl")
plot_BO(axs[1], "paper/out/sol_gpt4_100.pkl", "Topk - GPT-4",
         raw_data[y_name], "LogS solubility", lim, label=False, data_file_random="paper/out/sol - random.pkl")
plot_BO(axs[2], "paper/out/sol_GPR_100.pkl", "GPR",
         raw_data[y_name], "LogS solubility", lim, label=True, data_file_random="paper/out/sol - random.pkl")

fig.legend(loc='upper center', bbox_to_anchor=(0.5,0),
          fancybox=True, shadow=True, ncol=6)
plt.savefig(f"figs/BO_sol", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
d_davinci = cloudpickle.load(open("paper/out/sol_davinci_100.pkl", "rb"))
d_gpt4 = cloudpickle.load(open("paper/out/sol_gpt4_100.pkl", "rb"))
d_gpr = cloudpickle.load(open("paper/out/sol_GPR_100.pkl", "rb"))

print(d_davinci['expected_improvement'][:, -1, 1].astype(float))
best_davinci = d_davinci['expected_improvement'][:, :, 1].astype(float).mean(axis=0)[-1]
print(f"DaVinci is top{np.sum(raw_data[y_name] > best_davinci)}: {best_davinci}")

print(d_gpt4['expected_improvement'][:, -1, 1].astype(float))
best_gpt4 = d_gpt4['expected_improvement'][:, :, 1].astype(float).mean(axis=0)[-1]
print(f"Gpt4 is top{np.sum(raw_data[y_name] > best_gpt4)}: {best_gpt4}")

print(d_gpr['expected_improvement'][:, -1, 1].astype(float))
best_gpr = d_gpr['expected_improvement'][:, :, 1].astype(float).mean(axis=0)[-1]
print(f"GPR is top{np.sum(raw_data[y_name] > best_gpr)}: {best_gpr}")

sns.histplot(raw_data[y_name])
plt.axvline(best_davinci, color='C1', linestyle='--', label="Davinci")
plt.axvline(best_gpt4, color='C2', linestyle='--', label="GPT4")
plt.axvline(best_gpr, color='C3', linestyle='--', label="GPR")
plt.legend()
plt.savefig(f"figs/hist_sol", dpi=300, bbox_inches='tight')
plt.show()

print(raw_data[raw_data[y_name] > best_davinci-0.08])



# BayesOpt experiments

## Utils

In [4]:
import uncertainty_toolbox as uct

def combine(s, l):
  '''Number of combinations of l elements with max = s'''
  return (s**l - (s-1)**(l))

def prob(s, l, n):
  '''Probability of getting a sample with max([x0,x1,...,xl]) = s where xi={0,n}'''
  return combine(s,l) * ((1/n)**l)

def expected_value_p(l, n):
  '''Expected value of max([x0,x1,...,xl]) where xi={0,n}'''
  E = [s * prob(s, l, n) for s in range(1,100+1)]
  return sum(E)

def expected_value_q(l, n, data):
  '''Expected value of max([x0,x1,...,xl]) where xi={0,n}'''
  quants = [data.quantile(i/100) for i in range(100+1)]
  # E = [(quants[s-1]) * prob(s, l, n) for s in range(1,100+1)]
  E = [((quants[s-1]+quants[s])/2) * prob(s, l, n) for s in range(1,100+1)]
  return sum(E)

from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)

# @retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def run_experiment(
    asktell, pool, raw_data, indexes, x_name, y_name, N=10, initial_train=1, ask_K=1, aq="random", start_index=0, calibrate=False
):
    if aq=='random_mean':
       return [ (i, expected_value_q(i, 100, raw_data[y_name])) for i in range(1,N+1) ]
    for i in indexes[:initial_train]:
        asktell.tell(raw_data[x_name].iloc[i], float(raw_data[y_name].iloc[i]))
    if calibrate:
        # y = [float(raw_data[y_name].iloc[i]) for i in indexes[:initial_train]]
        # pred = asktell.predict(y)
        # ymeans = np.array([yhi.mean() for yhi in pred])
        # ystds = np.array([yhi.std() for yhi in pred])
        # calibration_factor = uct.recalibration.optimize_recalibration_ratio(ymeans, ystds, np.array(y), criterion="miscal")
        calibration_factor = 5.0
        asktell.set_calibration_factor(calibration_factor)

    x = raw_data[x_name].tolist()

    pool.reset()
    xi = x[start_index]
    x.remove(xi)
    pool.choose(xi)
    yi = float(raw_data[raw_data[x_name] == xi][y_name].iloc[0])
    asktell.tell(xi, yi)
    point = [(xi, 1+initial_train, yi, yi)]
    best = point[0][-1]
    for i in range(1, N):
        if i == N - 1 and aq != "random":
            aq = "greedy"
        px, _, py = asktell.ask(pool, 
                                k=ask_K, 
                                aq_fxn=aq, 
                                _lambda=1.0, 
                                inv_filter=16,
                                aug_random_filter=0,
                                )
        for j in range(ask_K):
          xc = px[j]
          x.remove(xc)
          pool.choose(xc)
          y = float(raw_data[raw_data[x_name] == xc][y_name].iloc[0])
          asktell.tell(xc, y)
          best = max(y, best)
        point.append((xc, 1+initial_train+i*ask_K, best, y))
    return point

In [5]:
def get_dataset(data: str, M=5):
    match data:
        case "in-house":
            data_path = "./dataset/data/71023_BO_ready_pool.csv"
            raw_data = pd.read_csv(data_path)

            raw_data['Catalyst'] = raw_data['Prompt'].str.extract(r'(\b[A-Z][a-z]?:[A-Z][a-z]?:[A-Z][a-z]?\b)')
            unique_cat = raw_data['Catalyst'].unique()
            c = {c: 0.2+m*(5/len(unique_cat)) for m, c in enumerate(unique_cat)}
            raw_data['dummy_Completion'] = raw_data['Catalyst'].apply(lambda x: np.random.normal(c[x], 0.05))

            x_name = "Prompt"
            y_name = "dummy_Completion"
        case "ocm":
            data_path = "./dataset/data/12744_ocm_dataset.csv"
            raw_data = pd.read_csv(data_path, sep=";")
            raw_data = raw_data.sample(frac=1).reset_index(drop=True)
            x_name = "prompt"
            y_name = "completion"
        case "sol":
            data_path = "./dataset/data/esol_iupac.csv"
            raw_data = pd.read_csv(data_path)
            raw_data = raw_data.dropna()
            raw_data = raw_data[["IUPAC", "measured log(solubility:mol/L)"]]
            raw_data = raw_data.sample(frac=1).reset_index(drop=True)
            x_name = "IUPAC"
            y_name = "measured log(solubility:mol/L)"
        case _:
            raise ValueError("Unknown data")
        

    N = raw_data.shape[0]
    indexes = np.random.choice(raw_data.shape[0], int(N), replace=False)

    print(f"Dataset size: \n\t{N}")
    starts = raw_data.sort_values(by=y_name, ascending=True).head(100).sample(M)# np.random.randint(0, len(indexes), M)
    print(f"Start xs: \n\t{starts[x_name].to_list()}")
    print(f"Start ys: \n\t{starts[y_name].to_list()}")
    starts = starts.index
    print(f"Start indexes: \n\t{starts}\n")

    return raw_data, starts, indexes, x_name, y_name


In [6]:
def get_asktell(model: str, kwargs: dict = {}, pool: bolift.Pool = None, knn: int = 1):
    match model:
        case "instruct":
            kwargs['model']="gpt-3.5-turbo-instruct"
            return bolift.AskTellFewShotTopk(**kwargs)
        case "gpt-turbo":
            kwargs['model']="gpt-3.5-turbo"
            return bolift.AskTellFewShotTopk(**kwargs)
        case "gpt-4":
            kwargs['model']="gpt-4"
            return bolift.AskTellFewShotTopk(**kwargs)
        case "davinci":
            kwargs['model']="text-davinci-003"
            return bolift.AskTellFewShotTopk(**kwargs)
        case "curie":
            kwargs['model']="text-curie-001"
            return bolift.AskTellFewShotTopk(**kwargs)
        case "gpr":
            kwargs['pool'] = pool if pool else None
            kwargs['n_components'] = 32
            return bolift.AskTellGPR(**kwargs)
        case "knn":
            del kwargs['selector_k']
            kwargs['knn'] = knn
            return bolift.AskTellNearestNeighbor(**kwargs)
        case "krr":
            kwargs['alpha'] = 0.5
            return bolift.AskTellRidgeKernelRegression(**kwargs)
        case _:
            raise ValueError("Unknown model")

In [7]:
def read_bkp(path, pool_path, indexes, kwargs):
    if os.path.exists(pool_path):
        with open(pool_path, "rb") as f:
            pool = cloudpickle.load(f)
        pool.reset()
    else:
        x = [raw_data[x_name].iloc[i] for i in indexes]
        pool = bolift.Pool(list(x), formatter=kwargs['x_formatter'])
        cloudpickle.dump(pool, open(pool_path, "wb"))

    if os.path.exists(path):
        bayesOpts = cloudpickle.load(open(path, "rb"))
    else:
        bayesOpts = {}
    return bayesOpts, pool

## Config BO

In [15]:
initial_train = 0
ask_K = 5
N=20
M=5
model="krr"

### in-house

In [10]:
np.random.seed(0)
dataset="in-house"
kwargs = dict(
    prefix="",
    prompt_template=PromptTemplate(
        input_variables=["x", "y", "y_name"],
        template="Q: What is the {y_name} of {x}?@@@\nA: {y}###",
    ),
    suffix="What is the {y_name} of {x}?@@@\nA:",
    x_formatter=lambda x: f"experimental procedure: {x}",
    y_name="C2 yield",
    y_formatter=lambda y: f"{y:.2f}",
    selector_k=5,
    temperature=0.7
)

path = f"./out/{dataset}_{model}_42000_{initial_train}_{ask_K}_16nr.pkl"
pool_path = "./dataset/data/42000_in-house_pool.pkl"


### OCM

In [9]:
np.random.seed(0)
dataset="ocm"
kwargs = dict(
    prefix="",
    prompt_template=PromptTemplate(
        input_variables=["x", "y", "y_name"],
        template="Q: What is the {y_name} of {x}?@@@\nA: {y}###",
    ),
    suffix="What is the {y_name} of {x}?@@@\nA:",
    x_formatter=lambda x: f"experimental procedure: {x}",
    y_name="C2 yield",
    y_formatter=lambda y: f"{y:.2f}",
    selector_k=5,
    temperature=0.7
)

path = f"./out/{dataset}_{model}_12744_{initial_train}_{ask_K}_16nr.pkl"
pool_path = "./dataset/data/12744_ocm_pool.pkl"


### Solubility

In [16]:
np.random.seed(0)
dataset="sol"
kwargs = dict(
    prefix="",
    prompt_template=PromptTemplate(
        input_variables=["x", "y", "y_name"],
        template="Q: What is the {y_name} of {x}?@@@\nA: {y}###",
    ),
    suffix="What is the {y_name} of {x}?@@@\nA:",
    x_formatter=lambda x: f"iupac name {x}",
    y_name="measured log solubility in mols per litre",
    y_formatter=lambda y: f"{y:.2f}",
    selector_k=5,
    temperature=0.7,
)

path = f"./out/{dataset}_{model}_882_{initial_train}_{ask_K}_16nr.pkl"
pool_path = "./out/sol_pool.pkl"


## Run BO

In [12]:
import warnings
warnings.filterwarnings('ignore', message='Changing the sparsity structure of a csr_matrix is expensive.*')
warnings.filterwarnings('ignore', message='Input data is not contained to the unit cube.*')
warnings.filterwarnings('ignore', message='Input data is not standardized.*')
warnings.filterwarnings('ignore', message="Keyword arguments .* will be ignored because they are not allowed parameters for function .*", category=UserWarning)

raw_data, starts, indexes, x_name, y_name= get_dataset(dataset, M=M)
bayesOpts, pool = read_bkp(path, pool_path, indexes, kwargs)

asktell = get_asktell(model, kwargs=kwargs, pool=bolift.Pool(list(pool.sample(200))), knn=5)

Dataset size: 
	882
Start xs: 
	['1-methoxy-4-[2,2,2-trichloro-1-(4-methoxyphenyl)ethyl]benzene', '1,2,4,5-tetrachlorobenzene', '(2,3,5,6-tetrafluoro-4-methylphenyl)methyl 3-(2-chloro-3,3,3-trifluoroprop-1-enyl)-2,2-dimethylcyclopropane-1-carboxylate', '5-tert-butyl-3-(2,4-dichloro-5-propan-2-yloxyphenyl)-1,3,4-oxadiazol-2-one', 'N-ethyl-N-(2-methylprop-2-enyl)-2,6-dinitro-4-(trifluoromethyl)aniline']
Start ys: 
	[-6.89, -5.56, -7.321, -5.696, -6.124]
Start indexes: 
	Index([370, 21, 847, 785, 724], dtype='int64')



In [14]:
import warnings

warnings.filterwarnings('ignore', message='Changing the sparsity structure of a csr_matrix is expensive.*')
warnings.filterwarnings('ignore', message='Input data is not contained to the unit cube.*')
warnings.filterwarnings('ignore', message='Input data is not standardized.*')
warnings.filterwarnings('ignore', message="Keyword arguments .* will be ignored because they are not allowed parameters for function .*", category=UserWarning)

for aq in ['random_mean', 'expected_improvement', 'random']: #["random", "expected_improvement", "greedy", 'upper_confidence_bound', 'probability_of_improvement']:
    print(aq, "start:", end=" ")
    points = []
    for i in range(M):
        print(i, end=",  ")
        point = run_experiment(
            copy.deepcopy(asktell),
            copy.deepcopy(pool),
            raw_data,
            indexes,
            x_name,
            y_name,
            N=N,
            aq=aq,
            start_index=starts[i],
            calibrate=True,
            initial_train=initial_train,
            ask_K=ask_K
        )
        points.append(point)
    points = np.array(points)
    bayesOpts[aq] = points
    print(aq, "done")
    if isinstance(asktell, bolift.AskTellGPR):
        asktell.save_cache("GPR_ada_embed_cache.csv")
    cloudpickle.dump(bayesOpts, open(path, "wb"))
cloudpickle.dump(bayesOpts, open(path, "wb"))


random_mean start: 0,  1,  2,  3,  4,  random_mean done
random start: 0,  1,  2,  3,  4,  random done
expected_improvement start: 0,  

AttributeError: 'AskTellNearestNeighbor' object has no attribute 'inv_prompt'

In [11]:
title = f"{path[6:-4]}"
d = cloudpickle.load(open(path, "rb"))
data=raw_data[y_name]
lim=(data.min()-1, data.max()+1)

# name = "LogS"
name = "C$_2$ yield"


def plot_config():
    plt.title(title)
    plt.axhline(y=data.max(), color="C15", linestyle="--")
    plt.text(plt.xlim()[1]+1, data.max(), "max", va="center", ha="left", backgroundcolor="w", fontsize=8)
    plt.axhline(y=data.quantile(0.99), color="C14", linestyle="--")
    plt.text(plt.xlim()[1]+1, data.quantile(0.99), "99%", va="center", ha="left", backgroundcolor="w", fontsize=8)
    plt.axhline(y=data.quantile(0.95), color="C13", linestyle="--")
    plt.text(plt.xlim()[1]+1, data.quantile(0.95), "95%", va="center", ha="left", backgroundcolor="w", fontsize=8)
    plt.axhline(y=data.mean(), color="C12", linestyle="--")
    plt.text(plt.xlim()[1]+1, data.mean(), "mean", va="center", ha="left", backgroundcolor="w", fontsize=8)
    plt.axhline(y=data.quantile(0.05), color="C11", linestyle="--")
    plt.text(plt.xlim()[1]+1, data.quantile(0.05), "5%", va="center", ha="left", backgroundcolor="w", fontsize=8)
    plt.axhline(y=data.min(), color="C10", linestyle="--")
    plt.text(plt.xlim()[1]+1, data.min(), "Min", va="center", ha="left", backgroundcolor="w", fontsize=8)
    plt.ylim(lim)
    plt.legend(loc='upper center', bbox_to_anchor=(0.5,-0.1),
          fancybox=True, shadow=True, ncol=5)

#Debugging plots
fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(6,4), constrained_layout=True)
plot_BO(axs, path, title, 
         raw_data[y_name], name, lim, label=True)
fig.legend(loc='upper center', bbox_to_anchor=(0.5,0),
          fancybox=True, shadow=True, ncol=6)
plt.show()

# Plot best value on the entire run
plt.figure(figsize=(8,5))
plt.xlabel("Number of samples")
# plt.ylabel("Max C$_2$ yield")
plt.ylabel(f"Max {name}")
for i in range(M):
    plt.plot(d['expected_improvement'][i,:,1], d['expected_improvement'][i, :, 2].astype(float), label=f"run {i}")
# plt.plot(d["expected_improvement"][i,:,1], d["expected_improvement"][:, :, 2].astype(float).mean(axis=0), label=f"EI", color="C1")
# plt.plot(d["random"][i,:,1], d["random"][:, :, 2].astype(float).mean(axis=0), label=f"random", color="C5")
plot_config()
plt.show()

# Plot current values on each iteration
plt.figure(figsize=(8,5))
plt.xlabel("Number of samples")
plt.ylabel(f"{name}")
for i in range(M):
    plt.plot(d['expected_improvement'][i,:,1], d['expected_improvement'][i, :, 3].astype(float), label=f"run {i}")
# plt.plot(d["expected_improvement"][i,:,1], d["expected_improvement"][:, :, 3].astype(float).mean(axis=0), label=f"EI", color="C1")
# plt.plot(d["random"][i,:,1], d["random"][:, :, 3].astype(float).mean(axis=0), label=f"random", color="C5")
plot_config()
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: './out/ocm_gpr_12744_0_5_16nr.pkl'

# Legacy

### in-house

In [29]:
def get_asktell():
  return bolift.AskTellFewShotTopk(
    # prefix="You are a bot who knows chemistry and catalysts. " \
    #       "Below, you'll see examples of experimental procedures to synthesize catalysts and the measured CO STY in a oxidative methane coupling reaction. " \
    #       "The following question should be answered with a number and finished with ###\n",
    prefix="",
    prompt_template=PromptTemplate(
        input_variables=["x", "y", "y_name"],
        template="Q: What is the {y_name} of {x}?@@@\nA: {y}###",
    ),
    suffix="What is the {y_name} of {x}?@@@\nA:",
    x_formatter=lambda x: f"experimental procedure: {x}",
    y_name="C2 yield",
    y_formatter=lambda y: f"{y:.2f}",
    # model="text-curie-001",
    # model="text-davinci-003",
    model = "gpt-3.5-turbo-instruct",
    # model="gpt-4",
    selector_k=5,
    temperature=0.7,
  )

In [30]:
np.random.seed(0)
data_path = "./dataset/data/71023_BO_ready_pool.csv"
path_random = "./out/in-house - random - 42000.pkl"
# template:    db_model_dbFilter_initial_pool
path = "./out/in-house_davinci_42000_0_16nr_krandom.pkl"
pool_path = "./dataset/data/42000_in-house_pool.pkl"

initial_train = 0
ask_K = 1
raw_data = pd.read_csv(data_path)

raw_data['Catalyst'] = raw_data['Prompt'].str.extract(r'(\b[A-Z][a-z]?:[A-Z][a-z]?:[A-Z][a-z]?\b)')
unique_cat = raw_data['Catalyst'].unique()
c = {c: 0.2+m*(5/len(unique_cat)) for m, c in enumerate(unique_cat)}
raw_data['dummy_Completion'] = raw_data['Catalyst'].apply(lambda x: np.random.normal(c[x], 0.05))

N = raw_data.shape[0]
indexes = np.random.choice(raw_data.shape[0], int(N), replace=False)
x_name = "Prompt"
y_name = "dummy_Completion"
print(N, len(indexes))

if os.path.exists(pool_path):
  with open(pool_path, "rb") as f:
    pool = cloudpickle.load(f)
  pool.reset()
else:
  x = [raw_data[x_name].iloc[i] for i in indexes]
  pool = bolift.Pool(list(x), formatter=lambda x: f"experimental procedure: {x}")
  cloudpickle.dump(pool, open(pool_path, "wb"))

N = 20
M = 5
starts = raw_data.sort_values(by=y_name, ascending=True).head(100).sample(M)# np.random.randint(0, len(indexes), M)
print(starts)
starts = starts.index
print(starts)


42000 42000
                                                  Prompt  Completion  \
25207  2.0 g of Fe:Ni:Zn(1:10:5)/ZrO2 catalyst is pro...         NaN   
19621  2.0 g of Fe:Ni:Zn(10:5:5)/MgO catalyst is prod...         NaN   
35006  2.0 g of Fe:Ni:Zn(1:10:1)/Al2O3 catalyst is pr...         NaN   
32209  2.0 g of Fe:Ni:Zn(5:1:1)/ZrO2 catalyst is prod...         NaN   
9817   2.0 g of Fe:Ni:Zn(10:1:1)/TiO2 (rutile) cataly...         NaN   

       Catalyst  dummy_Completion  
25207  Fe:Ni:Zn          0.117423  
19621  Fe:Ni:Zn          0.128967  
35006  Fe:Ni:Zn          0.123826  
32209  Fe:Ni:Zn          0.094579  
9817   Fe:Ni:Zn          0.130724  
Index([25207, 19621, 35006, 32209, 9817], dtype='int64')


In [ ]:
if os.path.exists(path):
    bayesOpts_random = cloudpickle.load(open(path_random, "rb")) #random_mean
    bayesOpts = cloudpickle.load(open(path, "rb"))
else:
    bayesOpts = {}

for aq in ['random', 'expected_improvement']: #["random", "expected_improvement", "greedy", 'upper_confidence_bound', 'probability_of_improvement']:
    print(aq, "start:", end=" ")
    points = []
    for i in range(M):
        asktell = get_asktell()
        print(i, end=",  ")
        point = run_experiment(
            copy.deepcopy(asktell),
            copy.deepcopy(pool),
            raw_data,
            indexes,
            x_name,
            y_name,
            N=N,
            aq=aq,
            start_index=starts[i],
            calibrate=True,
            initial_train=initial_train,
            ask_K=ask_K
        )
        points.append(point)
    points = np.array(points)
    bayesOpts[aq] = points
    print(aq, "done")
    # asktell.save_cache("GPR_ada_embed_cache.csv")
    cloudpickle.dump(bayesOpts, open(path, "wb"))
cloudpickle.dump(bayesOpts, open(path, "wb"))


In [ ]:
#Debugging plots
import re
path="./out/in-house_davinci_42000_0_16nr_krandom.pkl"
title = "in-house / no_random "
d = cloudpickle.load(open(path, "rb"))

fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(6,4), constrained_layout=True)
# for ax in axs.flat:
#     ax.set_aspect(0.6)
lim=(-0.5, 6)
plot_BO(axs, path, title, 
         raw_data[y_name], "C$_2$ yield", lim, label=True, data_file_random=path_random)
fig.legend(loc='upper center', bbox_to_anchor=(0.5,0),
          fancybox=True, shadow=True, ncol=6)
plt.savefig(f"figs/BO_C2", dpi=300, bbox_inches='tight')
plt.show()

plt.figure(figsize=(8,5))
plt.xlabel("Number of samples")
plt.ylabel("Catalyst mean STY")
plt.title(title)
for i in range(M):
    cats = [re.search(r'(\b[A-Z][a-z]?:[A-Z][a-z]?:[A-Z][a-z]?\b)',k).group(0) for k in d['expected_improvement'][i, :, 0]]
    print(cats)
    cats_ndx = [c[cat] for cat in cats]
    plt.plot(d['expected_improvement'][i,:,1], cats_ndx, label=f"run {i}")
plt.legend(loc='upper center', bbox_to_anchor=(0.5,-0.1),
          fancybox=True, shadow=True, ncol=5)
plt.show()


plt.figure(figsize=(8,5))
plt.xlabel("Number of samples")
plt.ylabel("Max catalyst mean STY")
plt.title(title)
for i in range(M):
    plt.plot(d['expected_improvement'][i,:,1], d['expected_improvement'][i, :, 2].astype(float), label=f"run {i}")
plt.legend(loc='upper center', bbox_to_anchor=(0.5,-0.1),
          fancybox=True, shadow=True, ncol=5)
plt.show()

plt.figure(figsize=(8,5))
plt.xlabel("Number of samples")
plt.ylabel("Catalyst mean STY")
plt.title(title)
for i in range(M):
    plt.plot(d['expected_improvement'][i,:,1], d['expected_improvement'][i, :, 3].astype(float), label=f"run {i}")
plt.legend(loc='upper center', bbox_to_anchor=(0.5,-0.1),
          fancybox=True, shadow=True, ncol=5)
plt.show()


In [31]:
x_name, y_name = "Prompt", "Completion"
raw_data_w_label = raw_data.dropna()
asktell = get_asktell()
asktell.tell(raw_data_w_label[x_name].iloc[0], float(raw_data_w_label[y_name].iloc[0]))
asktell.ask(pool, k=5, aq_fxn="expected_improvement", _lambda=1.0, inv_filter=16, aug_random_filter=0)

(['2.0 g of Zn:Mo:W(5:10:10)/MgO catalyst is produced by impregnating 1.9 g of MgO with zinc nitrate hexahydrate (0.0200 g, 0.306 mmol Zn), ammonium molybdate(para) tetrahydrate (0.0400 g, 0.417 mmol Mo), ammonium metatungstate hydrate (0.0400 g, 0.218 mmol W). The impregnated sample is heated to 90 °C, dried for four hours, then calcined at 450 °C for an additional four hours. A 25.0 mg portion of the calcined sample is pretreated at 450 °C for two hours under a flow of H2 (20 mL/min), then used for CO2 hydrogenation at 275 °C with a CO2/H2/Ar mixture (22.2%/66.7%/11.1%) flowing at 45 mL/min.',
  '2.0 g of W:V:Mn(1:1:5)/ZrO2 catalyst is produced by impregnating 1.9 g of ZrO2 with ammonium metatungstate hydrate (0.0143 g, 0.078 mmol W), vanadium(iii) chloride (0.0143 g, 0.280 mmol V), manganese(ii) nitrate hydrate (0.0714 g, 1.300 mmol Mn). The impregnated sample is heated to 90 °C, dried for four hours, then calcined at 450 °C for an additional four hours. A 150.0 mg portion of the ca

In [37]:
import numpy as np
import re
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback
from langchain.cache import InMemoryCache
import langchain
from dataclasses import dataclass

from langchain.schema import HumanMessage, SystemMessage

np.random.seed(0)
data_path = "./dataset/data/71023_BO_ready_pool.csv"
path_random = "./out/in-house - random - 42000.pkl"
# template:    db_model_dbFilter_initial_pool
path = "./out/in-house_davinci_42000_0_16nr_pref.pkl"
pool_path = "./dataset/data/42000_in-house_pool.pkl"
initial_train = 1
ask_K = 1
raw_data = pd.read_csv(data_path)
N = raw_data.shape[0]
indexes = np.random.choice(raw_data.shape[0], int(N), replace=False)
x_name = "Prompt"
y_name = "dummy_Completion"

asktell=get_asktell()

raw_data['Catalyst'] = raw_data['Prompt'].str.extract(r'(\b[A-Z][a-z]?:[A-Z][a-z]?:[A-Z][a-z]?\b)')
unique_cat = raw_data['Catalyst'].unique()
c = {c: 0.2+m*(5/len(unique_cat)) for m, c in enumerate(unique_cat)}
raw_data['dummy_Completion'] = raw_data['Catalyst'].apply(lambda x: np.random.normal(c[x], 0.05))

print(N, len(indexes))
for i in [1,123,6578,2134,-2]:#starts[2]:
    # print(raw_data[x_name].iloc[i], float(raw_data[y_name].iloc[i]))
    asktell.tell(raw_data[x_name].iloc[i], float(raw_data[y_name].iloc[i]))

def wrap_chatllm(query_list, llm):
    if type(llm) == ChatOpenAI:
        system_message_prompt = SystemMessage(
            content="You are a bot that can predict chemical and material properties. Do not explain answers, just provide numerical predictions."
        )
        if type(query_list) == str:
            query_list = [system_message_prompt, HumanMessage(content=query_list)]
        else:
            query_list = [
                [system_message_prompt, HumanMessage(content=q)] for q in query_list
            ]
    return query_list

# asktell.inv_predict(y=15)
query = asktell.inv_prompt.format(
            y=asktell.format_y(15.0), y_name=asktell._y_name, x_name=asktell._x_name
    )

print(query)
print()

print(
    asktell.inv_llm(
        wrap_chatllm(query, asktell.inv_llm)
        )
    )

# print(asktell.prompt.format(x=asktell.format_x("a given procedure"), y_name=asktell._y_name))

42000 42000
If C2 yield is 2.78, then input is @@@
experimental procedure: 2.0 g of Ni:Cu:Mn(5:1:1)/SiO2 catalyst is produced by impregnating 1.9 g of SiO2 with nickel(ii) nitrate hexahydrate (0.0714 g, 1.217 mmol Ni), copper(ii) nitrate hydrate (0.0143 g, 0.225 mmol Cu), manganese(ii) nitrate hydrate (0.0143 g, 0.260 mmol Mn). The impregnated sample is heated to 90 °C, dried for four hours, then calcined at 450 °C for an additional four hours. A 25.0 mg portion of the calcined sample is pretreated at 450 °C for two hours under a flow of H2 (20 mL/min), then used for CO2 hydrogenation at 275 °C with a CO2/H2/Ar mixture (22.2%/66.7%/11.1%) flowing at 45 mL/min.###

If C2 yield is 5.09, then input is @@@
experimental procedure: 2.0 g of W:V:Mn(10:10:1)/MgO catalyst is produced by impregnating 1.9 g of MgO with ammonium metatungstate hydrate (0.0476 g, 0.259 mmol W), vanadium(iii) chloride (0.0476 g, 0.935 mmol V), manganese(ii) nitrate hydrate (0.0048 g, 0.087 mmol Mn). The impregnated s

### C2

In [ ]:
# asktell = bolift.AskTellFewShotMulti(
#   x_formatter=lambda x: f"experimental procedure: {x}",
#   y_name="C2 yield",
#   y_formatter=lambda y: f"{y:.2f}",
#   model="text-curie-001",
#   selector_k=5,
#   temperature=0.05
# )

# Choose a Name from the list. Use the last part of the name to determine which 'sup' to use. Use 'sup' as the support and remember it should always be "(1.0 g)".
# Choose one from mol1 as M1. The corresponding value is always from 'm2_mol' at the first position. Choose one from mol2 as M2. The corresponding value is always from 'm2_mol' at the second position. Choose one from mol3 as M3. The corresponding value is always from 'm3_mol' at the first position. Select a temperature from Temp. Choose three different flow rates: one from 'ArVol_F' for Ar, one from 'CH4Vol_F' for CH4, and one from 'O2Vol_F' for O2. Ensure their sum equals a value from 'Vol_T' chosen. Based on the sum from step 7, choose a corresponding contact time from 'cont_time'.

asktell = bolift.AskTellFewShotTopk(
#   prefix="You are a bot who knows chemistry and catalysts. " \
#         "Below, you'll see examples of experimental procedures to synthesize catalysts and the measured C2 yield in a oxidative methane coupling reaction. " \
#         "The following question should be answered with a number and finished with ###\n",
#   inv_prefix='''
# Name =['Mn-Na2WO4/BN', 'Mn-Na2WO4/MgO', 'Mn-Na2WO4/Al2O3',   'Mn-Na2WO4/SiO2', 'Mn-Na2WO4/SiC', 'Mn-Na2WO4/SiCnf', 'Mn-Na2WO4/BEA', 'Mn-Na2WO4/ZSM-5', 'Mn-Na2WO4/TiO2', 'Mn-Na2WO4/ZrO2', 'Mn-Na2WO4/Nb2O5', 'Mn-Na2WO4/CeO2',   'Mn-Li2WO4/SiO2', 'Mn-MgWO4/SiO2', 'Mn-K2WO4/SiO2',   'Mn-CaWO4/SiO2', 'Mn-SrWO4/SiO2', 'Mn-BaWO4/SiO2',   'Mn-Li2MoO4/SiO2', 'Mn-Na2MoO4/SiO2', 'Mn-K2MoO4/SiO2',   'Mn-FeMoO4/SiO2', 'Mn-ZnMoO4/SiO2', 'Ti-Na2WO4/SiO2',   'V-Na2WO4/SiO2', 'Fe-Na2WO4/SiO2', 'Co-Na2WO4/SiO2',   'Ni-Na2WO4/SiO2', 'Cu-Na2WO4/SiO2', 'Zn-Na2WO4/SiO2',   'Y-Na2WO4/SiO2', 'Zr-Na2WO4/SiO2', 'Mo-Na2WO4/SiO2',   'Pd-Na2WO4/SiO2', 'La-Na2WO4/SiO2', 'Ce-Na2WO4/SiO2',   'Nd-Na2WO4/SiO2', 'Eu-Na2WO4/SiO2', 'Tb-Na2WO4/SiO2',   'Hf-Na2WO4/SiO2', 'Blank', 'BN', 'MgO', 'Al2O3', 'SiO2', 'SiC',   'SiCnf', 'BEA', 'ZSM-5', 'TiO2', 'ZrO2', 'Nb2O5', 'CeO2',   'Na2WO4/SiO2', 'Mn-WOx/SiO2', 'Mn-MoOx/SiO2', 'Mn-Na/SiO2',   'WOx/SiO2', 'Na/SiO2'] mol1 = ['Mn', 'Ti', 'V', 'Fe', 'Co', 'Ni', 'Cu', 'Zn', 'Y', 'Zr', 'Mo', 'Pd', 'La', 'Ce', 'Nd', 'Eu', 'Tb', 'Hf', 'n.a.'] mol2= ['Na', 'Li', 'Mg', 'K', 'Ca', 'Sr', 'Ba', 'Fe', 'Zn', 'n.a.'] mol3  =['W', 'Mo', 'n.a.'] sup =  ['BN', 'MgO', 'Al2O3', 'SiO2', 'SiC', 'SiCnf', 'BEA', 'ZSM-5','TiO2', 'ZrO2', 'Nb2O5', 'CeO2', 'n.a.'] m2_mol = [0.37 , 0.185, 0.  ] m3_mol = [0.185, 0.  ] Temp=[900, 850, 800, 775, 700, 750] Vol_T=  [10, 15, 20] ArVol_F = [ 1.5,  2.3,  3. ,  4. ,  6. ,  8. ,  7. , 10.5, 14. ] CH4Vol_F = [ 5.7,  8.5, 11.3,  6.4,  9.6, 12.8,  6.8, 10.2, 13.6,  7.3, 10.9,   14.6,  4. ,  6. ,  8. ,  4.5,  9. ,  4.8,  7.2,  5.1,  7.7, 10.3,   2. ,  3. ,  2.3,  3.4,  2.4,  3.6,  2.6,  3.9] O2Vol_F = [2.8, 4.3, 5.7, 2.1, 3.2, 1.7, 2.6, 3.4, 1.2, 1.8, 2.4, 2. , 3. , 4. , 1.5, 2.3, 0.9, 1.3, 1. , 0.8, 1.1, 0.6, 0.4] cont_time= [0.75, 0.5 , 0.38]
# These are the only parameters that you can choose from when selecting how to fill in the corresponding values within the paragraph template below.
  
# To synthesize (Name) , (sup) (1.0 g) was impregnated with 4.5 mL of an aqueous solution consisting of (M1) ( mol1) , M2 ( mol2) , M3 (mol3) , at 50 ºC for 6 h. Once activated the reaction is ran at (Temp) ºC. The total flow rate was (Vol_T)mL/min (Ar: (ArVol_F)  mL/min, CH4:(Vol_F) mL/min, O2: (Vol_F) mL/min), leading to a contact time of (cont_time) s.

# Can you give me eight new different, filled experimental procedures following these instructions and the examples below to answer the following question?
# Finish each procedure with ###.
# You can select parameters different from those you have seen before
# ''',
  prefix=""
  prompt_template=PromptTemplate(
      input_variables=["x", "y", "y_name"],
      template="Q: What is the {y_name} of {x}?@@@\nA: {y}###",
  ),
  suffix="What is the {y_name} of {x}?@@@\nA:",
  x_formatter=lambda x: f"experimental procedure: {x}",
  y_name="C2 yield",
  y_formatter=lambda y: f"{y:.2f}",
  # model="text-curie-001",
  model="text-davinci-003",
  # model="gpt-4",
  selector_k=5,
  temperature=0.7,
)

# asktell = bolift.AskTellGPR(
#   prefix="The following question should be answered with a number\n",
#   prompt_template=PromptTemplate(
#       input_variables=["x", "y", "y_name"],
#       template="Q: What is the {y_name} of {x}?@@@\nA: {y}###",
#   ),
#   suffix="What is the {y_name} of {x}?@@@\nA:",
#   x_formatter=lambda x: f"experimental procedure: {x}",
#   y_name="C2 yield",
#   y_formatter=lambda y: f"{y:.2f}",
#   model='text-ada-001',
#   pool=bolift.Pool([raw_data[x_name].iloc[i] for i in range(1000)], formatter=lambda x: f"experimental procedure: {x}"),
#   n_components=32,
#   # cache_path="GPR_ada_embed_cache.csv"
# )

# asktell = bolift.AskTellRidgeKernelRegression(
#     prefix="The following question should be answered with a number\n",
#     prompt_template=PromptTemplate(
#         input_variables=["x", "y", "y_name"],
#         template="Q: What is the {y_name} of {x}?@@@\nA: {y}###",
#     ),
#     suffix="What is the {y_name} of {x}?@@@\nA:",
#     x_formatter=lambda x: f"iupac name {x}",
#     y_name="measured log solubility in mols per litre",
#     y_formatter=lambda y: f"{y:.2f}",
#     model="text-ada-001",
#     alpha=0.5
#   )

In [ ]:
# def get_asktell():
#     return bolift.AskTellFewShotTopk(
#     prefix="",
#     prompt_template=PromptTemplate(
#         input_variables=["x", "y", "y_name"],
#         template="Q: What is the {y_name} of {x}?@@@\nA: {y}###",
#     ),
#     suffix="What is the {y_name} of {x}?@@@\nA:",
#     x_formatter=lambda x: f"experimental procedure: {x}",
#     y_name="C2 yield",
#     y_formatter=lambda y: f"{y:.2f}",
#     # model="text-curie-001",
#     model="gpt-3.5-turbo-instruct",
#     # model="gpt-4",
#     selector_k=5,
#     temperature=0.7,
#     )

def get_asktell():
    return bolift.AskTellGPR(
    prefix="The following question should be answered with a number\n",
    prompt_template=PromptTemplate(
        input_variables=["x", "y", "y_name"],
        template="Q: What is the {y_name} of {x}?@@@\nA: {y}###",
    ),
    suffix="What is the {y_name} of {x}?@@@\nA:",
    x_formatter=lambda x: f"experimental procedure: {x}",
    y_name="C2 yield",
    y_formatter=lambda y: f"{y:.2f}",
    model='text-ada-001',
    pool=bolift.Pool([raw_data[x_name].iloc[i] for i in range(1000)], formatter=lambda x: f"experimental procedure: {x}"),
    n_components=32,
    #   cache_path="GPR_ada_embed_cache.csv"
    )

In [ ]:
np.random.seed(88)

data_path = "./dataset/data/12744_ocm_dataset.csv"

path_random = "./out/C2 - random - 12744.pkl"
# template:    db_model_dbFilter_initial_pool
path = "./out/C2_GPR_12744_0_16nr.pkl"
pool_path = "./dataset/data/12744_ocm_pool_faiss.pkl"

initial_train = 0
ask_K = 1

raw_data = pd.read_csv(data_path, sep=";")

# raw_data['completion'] = - raw_data['completion']

N = raw_data.shape[0]
indexes = np.random.choice(raw_data.shape[0], int(N), replace=False)
x_name = "prompt"
y_name = "completion"
print(N, len(indexes))

if os.path.exists(pool_path):
  with open(pool_path, "rb") as f:
    pool = cloudpickle.load(f)
  pool.reset()
else:
  x = [raw_data[x_name].iloc[i] for i in indexes]
  pool = bolift.Pool(list(x), formatter=lambda x: f"experimental procedure: {x}")
  cloudpickle.dump(pool, open(pool_path, "wb"))

N = 20
M = 5
starts = raw_data.sort_values(by=y_name, ascending=True).head(100).sample(M)# np.random.randint(0, len(indexes), M)
print(starts)
starts = starts.index
print(starts)


In [ ]:
if os.path.exists(path):
    bayesOpts_random = cloudpickle.load(open(path_random, "rb")) #random_mean
    bayesOpts = cloudpickle.load(open(path, "rb"))
else:
    bayesOpts = {}

for aq in ['random', 'expected_improvement']: #["random", "expected_improvement", "greedy", 'upper_confidence_bound', 'probability_of_improvement']:
    print(aq, "start:", end=" ")
    points = []
    for i in range(M):
        asktell = get_asktell()
        print(i, end=",  ")
        point = run_experiment(
            copy.deepcopy(asktell),
            copy.deepcopy(pool),
            raw_data,
            indexes,
            x_name,
            y_name,
            N=N,
            aq=aq,
            start_index=starts[i],
            calibrate=True,
            initial_train=initial_train,
            ask_K=ask_K
        )
        points.append(point)
    points = np.array(points)
    bayesOpts[aq] = points
    print(aq, "done")
    # asktell.save_cache("GPR_ada_embed_cache.csv")
    cloudpickle.dump(bayesOpts, open(path, "wb"))
cloudpickle.dump(bayesOpts, open(path, "wb"))


In [ ]:
#Debugging plots
import re
path = "./out/C2_GPR_12744_0_16nr.pkl"
title = "OCM / no_random"
d = cloudpickle.load(open(path, "rb"))
aq = "random"

data = raw_data[y_name]

fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(6,4), constrained_layout=True)
# for ax in axs.flat:
#     ax.set_aspect(0.6)
lim=(0, 25)
plot_BO(axs, path, title, 
         raw_data[y_name], "C$_2$ yield", lim, label=True, data_file_random=path_random)
fig.legend(loc='upper center', bbox_to_anchor=(0.5,0),
          fancybox=True, shadow=True, ncol=6)
plt.savefig(f"figs/BO_C2", dpi=300, bbox_inches='tight')
plt.show()

plt.figure(figsize=(8,5))
plt.xlabel("Number of samples")
plt.ylabel("Max C2 yield")
plt.title(title)
for i in range(5):
    plt.plot(d[aq][i,:,1], d[aq][i, :, 2].astype(float), label=f"run {i}")
plt.axhline(y=data.max(), color="C15", linestyle="--")
plt.text(plt.xlim()[1]+1, data.max(), "max", va="center", ha="left", backgroundcolor="w", fontsize=8)
plt.axhline(y=data.quantile(0.99), color="C14", linestyle="--")
plt.text(plt.xlim()[1]+1, data.quantile(0.99), "99%", va="center", ha="left", backgroundcolor="w", fontsize=8)
plt.axhline(y=data.quantile(0.95), color="C13", linestyle="--")
plt.text(plt.xlim()[1]+1, data.quantile(0.95), "95%", va="center", ha="left", backgroundcolor="w", fontsize=8)
plt.axhline(y=data.mean(), color="C12", linestyle="--")
plt.text(plt.xlim()[1]+1, data.mean(), "mean", va="center", ha="left", backgroundcolor="w", fontsize=8)
plt.legend(loc='upper center', bbox_to_anchor=(0.5,-0.1),
          fancybox=True, shadow=True, ncol=5)
plt.show()

plt.figure(figsize=(8,5))
plt.xlabel("Number of samples")
plt.ylabel("C2 yield")
plt.title(title)
# for i in range(5):
#     plt.plot(d[aq][i,:,1], d[aq][i, :, 3].astype(float), label=f"run {i}")
plt.plot(d["expected_improvement"][i,:,1], d["expected_improvement"][:, :, 3].astype(float).mean(axis=0), label=f"EI", color="C1")
plt.plot(d["random"][i,:,1], d["random"][:, :, 3].astype(float).mean(axis=0), label=f"random", color="C5")
plt.axhline(y=data.max(), color="C15", linestyle="--")
plt.text(plt.xlim()[1]+1, data.max(), "max", va="center", ha="left", backgroundcolor="w", fontsize=8)
plt.axhline(y=data.quantile(0.99), color="C14", linestyle="--")
plt.text(plt.xlim()[1]+1, data.quantile(0.99), "99%", va="center", ha="left", backgroundcolor="w", fontsize=8)
plt.axhline(y=data.quantile(0.95), color="C13", linestyle="--")
plt.text(plt.xlim()[1]+1, data.quantile(0.95), "95%", va="center", ha="left", backgroundcolor="w", fontsize=8)
plt.axhline(y=data.mean(), color="C12", linestyle="--")
plt.text(plt.xlim()[1]+1, data.mean(), "mean", va="center", ha="left", backgroundcolor="w", fontsize=8)
plt.legend(loc='upper center', bbox_to_anchor=(0.5,-0.1),
          fancybox=True, shadow=True, ncol=5)
plt.show()


In [ ]:
import seaborn as sns

d_davinci = cloudpickle.load(open(path, "rb"))
print(d_davinci['expected_improvement'][:, -1, -1].astype(float))
best_davinci = d_davinci['expected_improvement'][:, :, -1].astype(float).mean(axis=0)[-1]
print(f"DaVinci is top{np.sum(raw_data[y_name] > best_davinci)}: {best_davinci}")

sns.histplot(raw_data[y_name])
# print(np.sum(raw_data[y_name] > best))
plt.xlabel("measured C$_2$ yield")
plt.axvline(best_davinci, color='C1', linestyle='--', label="Davinci")
plt.legend()
plt.savefig(f"figs/hist_C2", dpi=300, bbox_inches='tight')
plt.show()

print(raw_data[raw_data[y_name] > best_davinci])



In [ ]:
d = cloudpickle.load(open(path, "rb"))
for k in range(5):
    print([k[14:k.find(",")] for k in d['expected_improvement'][k, :, 0]])

In [ ]:
import numpy as np
import re
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback
from langchain.cache import InMemoryCache
import langchain
from dataclasses import dataclass

from langchain.schema import HumanMessage, SystemMessage

np.random.seed(0) # 88
data_path = "./dataset/data/C2_yield_meth_oxy_short.csv"
path_random = "./out/C2 - random - 12744.pkl"
path = "./out/C2_davinci_12744_1_16hh.pkl"
pool_path = "./dataset/data/12744_ocm_pool.pkl"
initial_train = 1
ask_K = 1
# raw_data = pd.read_csv(data_path, sep=";")
raw_data = pd.read_csv(data_path)
N = raw_data.shape[0]
indexes = np.random.choice(raw_data.shape[0], int(N), replace=False)
x_name = "prompt"
y_name = "completion"
print(N, len(indexes))
for i in starts[1:2]:
    print(raw_data[x_name].iloc[i], float(raw_data[y_name].iloc[i]))
    asktell.tell(raw_data[x_name].iloc[i], float(raw_data[y_name].iloc[i]))

def wrap_chatllm(query_list, llm):
    if type(llm) == ChatOpenAI:
        system_message_prompt = SystemMessage(
            content="You are a bot that can predict chemical and material properties. Do not explain answers, just provide numerical predictions."
        )
        if type(query_list) == str:
            query_list = [system_message_prompt, HumanMessage(content=query_list)]
        else:
            query_list = [
                [system_message_prompt, HumanMessage(content=q)] for q in query_list
            ]
    return query_list

# asktell.inv_predict(y=15)
query = asktell.inv_prompt.format(
            y=asktell.format_y(15.0), y_name=asktell._y_name, x_name=asktell._x_name
    )

print(query)
print()

print(
    asktell.inv_llm(
        wrap_chatllm(query, asktell.inv_llm)
        )
    )

# print(asktell.prompt.format(x=asktell.format_x("a given procedure"), y_name=asktell._y_name))

### IUPAC-Solubility

In [ ]:
np.random.seed(0)

data_path = "./dataset/data/esol_iupac.csv"
raw_data = pd.read_csv(data_path)
raw_data = raw_data.dropna()
raw_data = raw_data[["IUPAC", "measured log(solubility:mol/L)"]]
raw_data = raw_data.sample(frac=1).reset_index(drop=True)

# raw_data['measured log(solubility:mol/L)'] = -raw_data['measured log(solubility:mol/L)']

print(raw_data.columns)

N = raw_data.shape[0]
indexes = [i for i in range(N)]  # np.random.choice(raw_data.shape[0], int(N), replace=False)
x_name = "IUPAC"
y_name = "measured log(solubility:mol/L)"
print(len(raw_data), len(indexes))

asktell = bolift.AskTellFewShotTopk(
  prefix="",
  prompt_template=PromptTemplate(
      input_variables=["x", "y", "y_name"],
      template="Q: What is the {y_name} of {x}?@@@\nA: {y}###",
  ),
  suffix="What is the {y_name} of {x}?@@@\nA:",
  x_formatter=lambda x: f"iupac name {x}",
  y_name="measured log solubility in mols per litre",
  y_formatter=lambda y: f"{y:.2f}",
  model="gpt-3.5-turbo-instruct",
  selector_k=5,
  temperature=0.7,
)

# asktell = bolift.AskTellFewShotMulti(
#     x_formatter=lambda x: f"iupac name {x}",
#     y_name="measured log solubility in mols per litre",
#     y_formatter=lambda y: f"{y:.2f}",
#     model="text-curie-001",
#     selector_k=5,
#     temperature=0.05
# )

# asktell = bolift.AskTellGPR(
#   prefix="The following question should be answered with a number\n",
#   prompt_template=PromptTemplate(
#       input_variables=["x", "y", "y_name"],
#       template="Q: What is the {y_name} of {x}?@@@\nA: {y}###",
#   ),
#   suffix="What is the {y_name} of {x}?@@@\nA:",
#   x_formatter=lambda x: f"iupac name {x}",
#   y_name="measured log solubility in mols per litre",
#   y_formatter=lambda y: f"{y:.2f}",
#   model='text-ada-001',
#   pool=bolift.Pool(raw_data[x_name].to_list(), formatter=lambda x: f"iupac name {x}"),
#   n_components=16,
# )

In [ ]:
np.random.seed(0)

x = [raw_data[x_name].iloc[i] for i in indexes]
path_random = "./out/sol - random.pkl"
path = "./out/sol_davinci_0.pkl"
pool_path = "./out/sol_pool.pkl"

if os.path.exists(pool_path):
  with open(pool_path, "rb") as f:
    pool = cloudpickle.load(f)
  pool.reset()
else:
  x = [raw_data[x_name].iloc[i] for i in indexes]
  pool = bolift.Pool(list(x), formatter=lambda x: f"experimental procedure: {x}")
  cloudpickle.dump(pool, open(pool_path, "wb"))

N = 15
M = 5
starts = np.random.randint(0, len(indexes), M)
# starts = [110, 374, 790, 365, 523, 119, 560, 199, 239, 694, 608, 850, 599, 405, 510, 514, 264, 266, 261, 294, 612]
# print([raw_data[y_name].iloc[i] for i in starts])

In [ ]:
if os.path.exists(path):
    bayesOpts_random = cloudpickle.load(open(path_random, "rb"))
    bayesOpts = cloudpickle.load(open(path, "rb"))
else:
    bayesOpts = {}

for aq in ["random", "expected_improvement"]: #["random", "expected_improvement", "greedy", 'upper_confidence_bound', 'probability_of_improvement']:
    print(aq, "start:", end=" ")
    points = []
    for i in range(M):
        print(i, end=",  ")
        point = run_experiment(
            copy.deepcopy(asktell),
            copy.deepcopy(pool),
            raw_data,
            indexes,
            x_name,
            y_name,
            N=N,
            aq=aq,
            start_index=starts[i],
            calibrate=True,
            initial_train=0
        )
        points.append(point)
    # plot mean
    points = np.array(points)
    bayesOpts[aq] = points
    print(aq, "done")
    # asktell.save_cache("GPR_ada_embed_cache.csv")
    cloudpickle.dump(bayesOpts, open(path, "wb"))
cloudpickle.dump(bayesOpts, open(path, "wb"))

In [ ]:
d = cloudpickle.load(open(path, "rb"))
d_r = cloudpickle.load(open(path_random, "rb"))
N=15
M=5
aq = "random"
title = "LogS GPR"


data = raw_data[y_name]

fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(6,4), constrained_layout=True)
# for ax in axs.flat:
#     ax.set_aspect(0.6)
lim=(-5, 2)
plot_BO(axs, path, title, 
         raw_data[y_name], "C$_2$ yield", lim, label=True, data_file_random=path_random)
fig.legend(loc='upper center', bbox_to_anchor=(0.5,0),
          fancybox=True, shadow=True, ncol=6)
plt.savefig(f"figs/BO_C2", dpi=300, bbox_inches='tight')
plt.show()

plt.figure(figsize=(8,5))
plt.xlabel("Number of samples")
plt.ylabel("Max LogS")
plt.title(title)
for i in range(5):
    plt.plot(d[aq][i,:,1], d[aq][i, :, 2].astype(float), label=f"run {i}")
plt.axhline(y=data.max(), color="C15", linestyle="--")
plt.text(plt.xlim()[1]+1, data.max(), "max", va="center", ha="left", backgroundcolor="w", fontsize=8)
plt.axhline(y=data.quantile(0.99), color="C14", linestyle="--")
plt.text(plt.xlim()[1]+1, data.quantile(0.99), "99%", va="center", ha="left", backgroundcolor="w", fontsize=8)
plt.axhline(y=data.quantile(0.95), color="C13", linestyle="--")
plt.text(plt.xlim()[1]+1, data.quantile(0.95), "95%", va="center", ha="left", backgroundcolor="w", fontsize=8)
plt.axhline(y=data.mean(), color="C12", linestyle="--")
plt.text(plt.xlim()[1]+1, data.mean(), "mean", va="center", ha="left", backgroundcolor="w", fontsize=8)
plt.legend(loc='upper center', bbox_to_anchor=(0.5,-0.1),
          fancybox=True, shadow=True, ncol=5)
plt.show()

plt.figure(figsize=(8,5))
plt.xlabel("Number of samples")
plt.ylabel("LogS")
plt.title(title)
# for i in range(5):
#     plt.plot(d[aq][i,:,1], d[aq][i, :, 3].astype(float), label=f"run {i}")
plt.plot(d["expected_improvement"][i,:,1], d["expected_improvement"][:, :, 3].astype(float).mean(axis=0), label=f"EI", color="C1")
plt.plot(d["random"][i,:,1], d["random"][:, :, 3].astype(float).mean(axis=0), label=f"random", color="C5")
plt.axhline(y=data.max(), color="C15", linestyle="--")
plt.text(plt.xlim()[1]+1, data.max(), "max", va="center", ha="left", backgroundcolor="w", fontsize=8)
plt.axhline(y=data.quantile(0.99), color="C14", linestyle="--")
plt.text(plt.xlim()[1]+1, data.quantile(0.99), "99%", va="center", ha="left", backgroundcolor="w", fontsize=8)
plt.axhline(y=data.quantile(0.95), color="C13", linestyle="--")
plt.text(plt.xlim()[1]+1, data.quantile(0.95), "95%", va="center", ha="left", backgroundcolor="w", fontsize=8)
plt.axhline(y=data.mean(), color="C12", linestyle="--")
plt.text(plt.xlim()[1]+1, data.mean(), "mean", va="center", ha="left", backgroundcolor="w", fontsize=8)
plt.legend(loc='upper center', bbox_to_anchor=(0.5,-0.1),
          fancybox=True, shadow=True, ncol=5)
plt.show()


In [ ]:
import seaborn as sns

d['expected_improvement'][1][-1][1]
best = d['expected_improvement'][:, :, 1].astype(float).mean(axis=0)[-1]
print(best)

sns.histplot(raw_data[y_name])
print(np.sum(raw_data[y_name] > best))
plt.axvline(best, color='red', linestyle='--')
plt.show()


### Alloy

In [ ]:
import pandas as pd
import numpy as np

np.random.seed(0)

data_path = "paper/data/yield_strength.csv"
raw_data = pd.read_csv(data_path)
# raw_data = raw_data.sample(frac=1).reset_index(drop=True)

print(raw_data.columns)

N = raw_data.shape[0]
indexes = np.random.choice(raw_data.shape[0], int(N), replace=False)
# shuffle test

print(N, len(indexes))

asktell = bolift.AskTellFewShotTopk(
    prefix="",
    prompt_template=PromptTemplate(
        input_variables=["x", "y", "y_name"],
        template="Q: What is the {y_name} of {x}?@@@\nA: {y}###",
    ),
    suffix="What is the {y_name} of {x}?@@@\nA:",
    # x_formatter=lambda x: f"alloy composition: {x}",
    y_name="yield strength",
    y_formatter=lambda y: f"{y:.2f}",
    model="gpt-4",
    selector_k=5,
)


x_name = "composition"
y_name = "yield strength"


In [ ]:
N = 10
M = 5
starts = np.random.randint(0, len(indexes), M)
plt.figure(figsize=(3.5, 3.5 / 1.2))
random_points = []
for i in range(M):
    point = run_experiment(
        asktell,
        raw_data,
        indexes,
        x_name,
        y_name,
        N=N,
        aq="random",
        start_index=starts[i],
    )
    random_points.append(point)
    plt.plot(range(N + 1), [y for x, y in point], color="C0", alpha=0.1)
# plot mean
random_points = np.array(random_points)
plt.plot(
    range(N + 1),
    random_points[:, :, 1].astype(float).mean(axis=0),
    color="C0",
    label="Random",
)

greedy_points = []
for i in range(M):
    point = run_experiment(
        asktell,
        raw_data,
        indexes,
        x_name,
        y_name,
        N=N,
        aq="greedy",
        start_index=starts[i],
    )
    greedy_points.append(point)
    plt.plot(range(N + 1), [y for x, y in point], color="C2", alpha=0.1)
# plot mean
greedy_points = np.array(greedy_points)
plt.plot(
    range(N + 1),
    greedy_points[:, :, 1].astype(float).mean(axis=0),
    color="C2",
    label="Greedy",
)

ei_points = []
for i in range(M):
    point = run_experiment(
        asktell,
        raw_data,
        indexes,
        x_name,
        y_name,
        N=N,
        aq="expected_improvement",
        start_index=starts[i],
    )
    ei_points.append(point)
    plt.plot(range(N + 1), [y for x, y in point], color="C1", alpha=0.1)
# plot mean
ei_points = np.array(ei_points)
plt.plot(
    range(N + 1), ei_points[:, :, 1].astype(float).mean(axis=0), color="C1", label="EI"
)

plt.axhline(y=raw_data["yield strength"].min(), color="C0", linestyle="--", label="min")
plt.axhline(
    y=raw_data["yield strength"].mean(), color="C1", linestyle="--", label="mean"
)
plt.axhline(y=raw_data["yield strength"].max(), color="C2", linestyle="--", label="max")
# give 5% quantiles
plt.axhline(
    y=raw_data["yield strength"].quantile(0.05), color="C3", linestyle="--", label="5%"
)
plt.axhline(
    y=raw_data["yield strength"].quantile(0.95), color="C4", linestyle="--", label="95%"
)

plt.xlabel("Samples")
# reduce number of ticks
# plt.xticks([0, 5, 10])
# plt.ylim(-10, 0)
# plt.yticks(np.linspace(-10, 0, 3))
plt.legend(loc="center left", bbox_to_anchor=(1.05, 0.5))
plt.savefig("concept_multi.png", dpi=300)